# Trabalho NLP - Desafio 01

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import re
import numpy as np
from time import time
from pprint import pprint

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

nltk.download('stopwords')
nltk.download('punkt')


from nltk.stem import SnowballStemmer

from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn import metrics


#modelos
from sklearn.linear_model import SGDClassifier



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mbjesus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mbjesus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Etapa 1 - Obter dados Rotulados

In [4]:
dataset = pd.read_csv("dataset/train.csv")
datasetTeste = pd.read_csv("dataset/test.csv")

In [5]:
datasetTeste["id"] = datasetTeste.index
datasetTeste.head(10)


,title,id
0,Strobo Mini Kit Com 5 Strobos,0
1,Filtro Hoya Hmc Uv(c) - Uv-hmc 72mm,1
2,Dvd Next Lite (envio Toda Segunda),2
3,Fim De Curso Tubinho Do Motor Desl Peccinin ...,3
4,Carro Infantil Smart Baby Reclinável Bandeiran...,4
5,Par Coifa Caixa Direção Hidraulica C4 Lounge 2...,5
6,Protetor Sofá Retrátil Reclinavél Com Assento ...,6
7,Pedal Behringer Tu300 Afinador Cromatico,7
8,Receptor Satmax Elsys,8
9,Mini Fantasia Noivinha,9


## Etapa 2 - Pré-processamento 

### 2.1 - Converter para minúsculo

In [6]:
dataset["category"] = dataset["category"].str.lower()
dataset["title"] = dataset["title"].str.lower()
dataset.head(5)


,title,category
0,tampa para champanhe cromado ibili - 722500,ice_buckets
1,copo térmico plástico os jetsons família espacial,thermal_cups_and_tumblers
2,nobreak sms bivolt 600va para pc ps4 xbox,stabilizers_and_ups
3,led de balada rgb luz de palco dj disco strobo...,led_stage_lights
4,filtro de lente cpl 52mm kenko polarizador par...,lens_filters


In [7]:
datasetTeste["title"] = datasetTeste["title"].str.lower()
datasetTeste.head(5)

,title,id
0,strobo mini kit com 5 strobos,0
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1
2,dvd next lite (envio toda segunda),2
3,fim de curso tubinho do motor desl peccinin ...,3
4,carro infantil smart baby reclinável bandeiran...,4


### 2.2 - Remover caractéres especiais

In [8]:
##FUNCAO QUE REMOVE CARACTERES ESPECIAIS
def remover_acentos(texto):
    
    ##REMOVER ACENTOS
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    
    ##REMOVER CARACTERES ESPECIAIS
    texto = re.sub(r'[?|$|&|*|%|@|(|)|~|,|\.|:|\-|\'|/|+]', r' ', texto)
    
    ##REMOVER MEDIDAS
    texto = re.sub('(va)|(mm)|(km)|(cm)', r' ', texto)
    
    ##REMOVER NUMEROS QUE APARECEM ISOLADOS
    texto = re.sub('[\s][0-9]{1,}', ' ', texto)
    
     ##REMOVER TEXTOS COM ATE DOIS CARACTERES
    texto = re.sub(r'\b\w{1,2}\b', ' ', texto)
    
    return texto



In [9]:
#aplica limpeza no texto
dataset["title_normalizado"] = dataset["title"].map(remover_acentos)
dataset.head(5)


,title,category,title_normalizado
0,tampa para champanhe cromado ibili - 722500,ice_buckets,tampa para champanhe cromado ibili
1,copo térmico plástico os jetsons família espacial,thermal_cups_and_tumblers,copo termico plastico jetsons familia espacial
2,nobreak sms bivolt 600va para pc ps4 xbox,stabilizers_and_ups,nobreak sms bivolt para ps4 xbox
3,led de balada rgb luz de palco dj disco strobo...,led_stage_lights,led balada rgb luz palco disco strobo fe...
4,filtro de lente cpl 52mm kenko polarizador par...,lens_filters,filtro lente cpl kenko polarizador para ob...


In [10]:
datasetTeste["title_normalizado"] = datasetTeste["title"].map(remover_acentos)
datasetTeste.head(5)

,title,id,title_normalizado
0,strobo mini kit com 5 strobos,0,strobo mini kit com strobos
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1,filtro hoya hmc hmc
2,dvd next lite (envio toda segunda),2,dvd next lite envio toda segunda
3,fim de curso tubinho do motor desl peccinin ...,3,fim curso tubinho motor desl peccinin li...
4,carro infantil smart baby reclinável bandeiran...,4,carro infantil smart baby reclinavel bandeiran...


### 2.3 - Remover Stop-words

In [11]:
def remove_stopwords(texto):
    #REMOVER STOPWORDS
    ##baixar stopwords
    stopwords = nltk.corpus.stopwords.words('portuguese')
    
    #gera tokens
    tokens = word_tokenize(texto)
    
    #remove stopwords
    filtered_tokens = [token.lower() for token in tokens if token not in stopwords]
    
    #retorna texto sem as stop-words
    return ' '.join(filtered_tokens)

In [12]:
#aplica remocao de stop-words
dataset["title_normalizado_sem_sw"] = dataset["title_normalizado"].map(remove_stopwords)
dataset.head(5)

,title,category,title_normalizado,title_normalizado_sem_sw
0,tampa para champanhe cromado ibili - 722500,ice_buckets,tampa para champanhe cromado ibili,tampa champanhe cromado ibili
1,copo térmico plástico os jetsons família espacial,thermal_cups_and_tumblers,copo termico plastico jetsons familia espacial,copo termico plastico jetsons familia espacial
2,nobreak sms bivolt 600va para pc ps4 xbox,stabilizers_and_ups,nobreak sms bivolt para ps4 xbox,nobreak sms bivolt ps4 xbox
3,led de balada rgb luz de palco dj disco strobo...,led_stage_lights,led balada rgb luz palco disco strobo fe...,led balada rgb luz palco disco strobo festa ba...
4,filtro de lente cpl 52mm kenko polarizador par...,lens_filters,filtro lente cpl kenko polarizador para ob...,filtro lente cpl kenko polarizador objeti


In [13]:
datasetTeste["title_normalizado_sem_sw"] = datasetTeste["title_normalizado"].map(remove_stopwords)
datasetTeste.head(5)

,title,id,title_normalizado,title_normalizado_sem_sw
0,strobo mini kit com 5 strobos,0,strobo mini kit com strobos,strobo mini kit strobos
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1,filtro hoya hmc hmc,filtro hoya hmc hmc
2,dvd next lite (envio toda segunda),2,dvd next lite envio toda segunda,dvd next lite envio toda segunda
3,fim de curso tubinho do motor desl peccinin ...,3,fim curso tubinho motor desl peccinin li...,fim curso tubinho motor desl peccinin light super
4,carro infantil smart baby reclinável bandeiran...,4,carro infantil smart baby reclinavel bandeiran...,carro infantil smart baby reclinavel bandeiran...


### 2.4 - Transformar para Radical

In [14]:
def stemmer_sentenca(texto):
    
    #baixa dados
    pt_stemmer = SnowballStemmer('portuguese')
    
    #divide em tokens
    tokens = word_tokenize(texto)
    
    #aplica stemmer
    lista_stem = [pt_stemmer.stem(token) for token in tokens ]
    
    #retorna texto 
    return ' '.join(lista_stem)

In [15]:
#aplica limpeza stemmer
dataset["title_normalizado_sem_sw_stemmer"] = dataset["title_normalizado_sem_sw"].map(stemmer_sentenca)
dataset.head(5)

,title,category,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
0,tampa para champanhe cromado ibili - 722500,ice_buckets,tampa para champanhe cromado ibili,tampa champanhe cromado ibili,tamp champanh crom ibil
1,copo térmico plástico os jetsons família espacial,thermal_cups_and_tumblers,copo termico plastico jetsons familia espacial,copo termico plastico jetsons familia espacial,cop termic plastic jetsons famil espacial
2,nobreak sms bivolt 600va para pc ps4 xbox,stabilizers_and_ups,nobreak sms bivolt para ps4 xbox,nobreak sms bivolt ps4 xbox,nobreak sms bivolt ps4 xbox
3,led de balada rgb luz de palco dj disco strobo...,led_stage_lights,led balada rgb luz palco disco strobo fe...,led balada rgb luz palco disco strobo festa ba...,led bal rgb luz palc disc strob fest bal
4,filtro de lente cpl 52mm kenko polarizador par...,lens_filters,filtro lente cpl kenko polarizador para ob...,filtro lente cpl kenko polarizador objeti,filtr lent cpl kenk polariz objet


In [16]:
datasetTeste["title_normalizado_sem_sw_stemmer"] = datasetTeste["title_normalizado_sem_sw"].map(stemmer_sentenca)
datasetTeste.head(5)

,title,id,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
0,strobo mini kit com 5 strobos,0,strobo mini kit com strobos,strobo mini kit strobos,strob min kit strob
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1,filtro hoya hmc hmc,filtro hoya hmc hmc,filtr hoy hmc hmc
2,dvd next lite (envio toda segunda),2,dvd next lite envio toda segunda,dvd next lite envio toda segunda,dvd next lit envi tod segund
3,fim de curso tubinho do motor desl peccinin ...,3,fim curso tubinho motor desl peccinin li...,fim curso tubinho motor desl peccinin light super,fim curs tubinh motor desl peccinin light sup
4,carro infantil smart baby reclinável bandeiran...,4,carro infantil smart baby reclinavel bandeiran...,carro infantil smart baby reclinavel bandeiran...,carr infantil smart baby reclinavel bandeir ros


### 2.5 - Balancear Dados com Uper-Undersampling

In [17]:
def balencearMesmaClasse(df_desbalanceado,tamanho):
        
    lista_categorias = df_desbalanceado.category.unique()
        
    df_sampled = pd.DataFrame()

    for categoria in lista_categorias:

        df_tmp = df_desbalanceado[df_desbalanceado.category == categoria]

        if(len(df_tmp)) > tamanho:
            ##como a classe tem mais registros que o tamanho, fazer SEM repeticao
            df_tmp = resample(df_tmp, 
                             replace=False,    
                             n_samples=(tamanho),    
                             random_state=123) 
        else:
            ##como a classe tem mais registros que o tamanho, fazer COM repeticao
            df_tmp = resample(df_tmp, 
                             replace=True,    
                             n_samples=(tamanho),    
                             random_state=123)

        df_sampled = pd.concat([df_sampled, df_tmp])

    return df_sampled

In [18]:
dataset_balanceado = balencearMesmaClasse(dataset,370)
dataset_balanceado.head(5)

,title,category,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
20135,balde de gelo acrílico p/ cerveja 5l baldinho ...,ice_buckets,balde gelo acrilico cerveja baldinho ge...,balde gelo acrilico cerveja baldinho gelo latas,bald gel acril cervej baldinh gel lat
3343,cervegela c/ refil de congelamento 2h mantém c...,ice_buckets,cervegela refil congelamento mantem cer...,cervegela refil congelamento mantem cerveja ge...,cervegel refil congel mant cervej gel
24862,balde brahma com alça e abridor de garrafas,ice_buckets,balde brahma com alca abridor garrafas,balde brahma alca abridor garrafas,bald brahm alca abridor garraf
14064,kit c/ 6 balde plástico reforçado plasnew 20 l...,ice_buckets,kit balde plastico reforcado plasnew litros,kit balde plastico reforcado plasnew litros,kit bald plastic reforc plasnew litr
27396,**balde brahma - luminoso de brinde!!!!,ice_buckets,balde brahma luminoso brinde!!!!,balde brahma luminoso brinde ! ! ! !,bald brahm lumin brind ! ! ! !


In [19]:
dataset_balanceado.shape

(30710, 5)

In [20]:
dataset.shape

(33500, 5)

In [21]:
###Separar os dados em treino e teste """
X_train, X_test, y_train, y_test = train_test_split(dataset_balanceado["title_normalizado_sem_sw_stemmer"], 
                                                    dataset_balanceado["category"], 
                                                    test_size=0.3, random_state = 0)

#CONSTRUIR MATRIZ ESPARSA """ 
count_vect = CountVectorizer(max_df= 0.5,
                            max_features= None,
                            min_df= 1,
                            ngram_range= (1, 2))

#fazer fit_transoform (vetor de palavras) para os dados de treino """
X_train_counts = count_vect.fit_transform(X_train)

#ponderar palavras mais frequentes  """
tf_transformer = TfidfTransformer(norm= 'l2',
                                  use_idf= True)
X_train_tf = tf_transformer.fit_transform(X_train_counts)

'''
Melhor score: 0.956
Melhores Parametros:
	clf__alpha: 1e-05
	clf__max_iter: 80
	clf__penalty: 'l2'
	tfidf__norm: 'l2'
	tfidf__use_idf: True
	vect__max_df: 0.5
	vect__max_features: None
	vect__min_df: 1
	vect__ngram_range: (1, 2)
       ''' 

"\nMelhor score: 0.956\nMelhores Parametros:\n\tclf__alpha: 1e-05\n\tclf__max_iter: 80\n\tclf__penalty: 'l2'\n\ttfidf__norm: 'l2'\n\ttfidf__use_idf: True\n\tvect__max_df: 0.5\n\tvect__max_features: None\n\tvect__min_df: 1\n\tvect__ngram_range: (1, 2)\n       "

## Etapa 3 - Aplicar aprendizado de máquina 

In [22]:
sgd = SGDClassifier(penalty='l2',
                    alpha=1e-05, 
                    max_iter=80)

#treinar modelo
t0 = time()
clf = sgd.fit(X_train_tf, y_train)
print("Fit done in %0.3fs" % (time() - t0))
y_pred_modelo = clf.predict(count_vect.transform(X_test))
print("Pred done in %0.3fs" % (time() - t0))

Fit done in 1.189s
Pred done in 1.339s


### Calcular desempenho 

In [23]:
print(metrics.classification_report(y_test, y_pred_modelo, target_names=dataset_balanceado["category"].unique()))


                                precision    recall  f1-score   support

                   ice_buckets       0.96      0.88      0.91        98
     thermal_cups_and_tumblers       0.95      0.94      0.95       104
           stabilizers_and_ups       0.94      0.94      0.94       110
              led_stage_lights       0.90      0.93      0.92       120
                  lens_filters       0.97      0.99      0.98       105
          engine_oil_dipsticks       0.93      0.94      0.93       105
                 roller_skates       0.98      0.96      0.97       113
                   pool_lights       0.92      0.89      0.91       110
                     key_racks       0.93      0.86      0.89       122
                   tv_antennas       0.91      0.92      0.91       111
            motorcycle_fenders       0.98      0.92      0.95       106
            soldering_machines       0.94      0.94      0.94       110
          motorcycle_batteries       0.91      0.99      0.95  

# Aplicar modelo nos dados de Teste

In [24]:
##Visao dos dados de Teste
datasetTeste.head(5)


,title,id,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
0,strobo mini kit com 5 strobos,0,strobo mini kit com strobos,strobo mini kit strobos,strob min kit strob
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1,filtro hoya hmc hmc,filtro hoya hmc hmc,filtr hoy hmc hmc
2,dvd next lite (envio toda segunda),2,dvd next lite envio toda segunda,dvd next lite envio toda segunda,dvd next lit envi tod segund
3,fim de curso tubinho do motor desl peccinin ...,3,fim curso tubinho motor desl peccinin li...,fim curso tubinho motor desl peccinin light super,fim curs tubinh motor desl peccinin light sup
4,carro infantil smart baby reclinável bandeiran...,4,carro infantil smart baby reclinavel bandeiran...,carro infantil smart baby reclinavel bandeiran...,carr infantil smart baby reclinavel bandeir ros


### Aplicando predição

In [25]:

y_pred_modelo_final = clf.predict(count_vect.transform(datasetTeste["title_normalizado_sem_sw_stemmer"]))


In [26]:
data_setPredito = pd.DataFrame(y_pred_modelo_final,columns=["category"])
data_setPredito.head(10)

,category
0,led_stage_lights
1,lens_filters
2,cd_and_dvd_players
3,gate_motors
4,push_and_riding_toys
5,power_steering_hoses
6,sofa_and_futon_covers
7,musical_tuners
8,tv_receivers_and_decoders
9,erotic_costumes


In [27]:
##Transformar em Data-Frame
data_setPredito['id'] = data_setPredito.index
data_setPredito['category'] = data_setPredito['category'].str.upper()
submissao = data_setPredito[["id","category"]] 

## Salvar no formato de submissao 
submissao.to_csv("submissao.csv", index = None, header=True)
submissao.head(10)


,id,category
0,0,LED_STAGE_LIGHTS
1,1,LENS_FILTERS
2,2,CD_AND_DVD_PLAYERS
3,3,GATE_MOTORS
4,4,PUSH_AND_RIDING_TOYS
5,5,POWER_STEERING_HOSES
6,6,SOFA_AND_FUTON_COVERS
7,7,MUSICAL_TUNERS
8,8,TV_RECEIVERS_AND_DECODERS
9,9,EROTIC_COSTUMES


### Gerando visualizacão amigável

In [29]:
visualizacaoResutado =   pd.merge(datasetTeste[["title","id"]],
                         data_setPredito[["id","category"]],
                         on='id')
visualizacaoResutado.head(200)


,title,id,category
0,strobo mini kit com 5 strobos,0,LED_STAGE_LIGHTS
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1,LENS_FILTERS
2,dvd next lite (envio toda segunda),2,CD_AND_DVD_PLAYERS
3,fim de curso tubinho do motor desl peccinin ...,3,GATE_MOTORS
4,carro infantil smart baby reclinável bandeiran...,4,PUSH_AND_RIDING_TOYS
5,par coifa caixa direção hidraulica c4 lounge 2...,5,POWER_STEERING_HOSES
6,protetor sofá retrátil reclinavél com assento ...,6,SOFA_AND_FUTON_COVERS
7,pedal behringer tu300 afinador cromatico,7,MUSICAL_TUNERS
8,receptor satmax elsys,8,TV_RECEIVERS_AND_DECODERS
9,mini fantasia noivinha,9,EROTIC_COSTUMES


# FIM 